In [ ]:
# Lab 2: RAG 기초 - 소방기본법 검색
# 소방청 민원처리 AI Agent 교육

"""
학습 목표:
- RAG(Retrieval Augmented Generation) 개념 이해
- PDF 텍스트 추출 및 청킹
- 임베딩과 벡터 검색
- ChromaDB 사용법
"""

# ============================================================
# 환경 설정
# ============================================================
import sys,os
if 'google.colab' in sys.modules:
    !pip install openai chromadb pdfplumber -q

import getpass
from openai import OpenAI
import pdfplumber
import chromadb
from chromadb.config import Settings
from typing import List, Dict

# API 키 설정
print("[API 키 입력] OpenAI API 키를 입력하세요")
api_key = "".strip()
client = OpenAI(api_key=api_key)

print(" 환경 설정 완료!\n")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.7/20.7 MB 97.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 111.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 81.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 99.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.5 MB/s eta 0:0

In [3]:

# ============================================================
# Lab 1 함수 재사용
# ============================================================

def classify_complaint(text: str) -> Dict[str, str]:
    """
    Lab 1에서 만든 민원 분류 함수
    """
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": """소방 민원을 분류하고 다음 형식으로 답변하세요:

카테고리: [화재신고/소방시설/법규위반/일반문의]
긴급도: [높음/보통/낮음]
키워드: [핵심 키워드 3개]
요약: [한 줄 요약]"""
            },
            {"role": "user", "content": text}
        ],
        temperature=0.3,
        max_tokens=200
    )

    result_text = response.choices[0].message.content

    return {
        "원문": text,
        "분류_결과": result_text
    }

# ============================================================
# 실습 1: PDF 텍스트 추출
# ============================================================

print("\n" + "="*60)
print("[실습 1] PDF 텍스트 추출")
print("="*60)

def extract_text_from_pdf(pdf_path: str, max_pages: int = None) -> List[Dict]:
    """
    PDF에서 텍스트 추출 (페이지별)
    pdfplumber 사용 - 한글 지원 우수
    """
    pages_data = []

    with pdfplumber.open(pdf_path) as pdf:
        total_pages = len(pdf.pages) if max_pages is None else min(max_pages, len(pdf.pages))

        print(f"[진행중] 총 {total_pages} 페이지 추출 중...")

        for i in range(total_pages):
            page = pdf.pages[i]
            text = page.extract_text()

            pages_data.append({
                "page_num": i + 1,
                "text": text,
                "char_count": len(text)
            })

            if (i + 1) % 5 == 0:
                print(f"  {i + 1}/{total_pages} 페이지 완료")

    return pages_data

# PDF 읽기
fire_law_pages = extract_text_from_pdf("소방기본법.pdf", max_pages=None)
pages = len(fire_law_pages)

print(f"\n[완료] 총 {pages} 페이지 추출")
print(f"\n[샘플] 1페이지 내용 (처음 300자):")
print(fire_law_pages[0]['text'][:300])
print("...")



[실습 1] PDF 텍스트 추출
[진행중] 총 16 페이지 추출 중...
  5/16 페이지 완료
  10/16 페이지 완료
  15/16 페이지 완료

[완료] 총 16 페이지 추출

[샘플] 1페이지 내용 (처음 300자):
발 간 등 록 번 호
소방기본법
2025. 6. 21
법제처 국가법령정보센터
Korea Law Service Center
...


In [4]:

# ============================================================
# 실습 2: 텍스트 청킹 (Chunking)
# ============================================================

print("\n\n" + "="*60)
print("[실습 2] 텍스트 청킹 - 작은 단위로 분할")
print("="*60)

def chunk_text(pages_data: List[Dict], chunk_size: int = 500, overlap: int = 50) -> List[Dict]:
    """
    텍스트를 작은 청크로 분할

    왜 청킹이 필요한가?
    - 전체 문서는 너무 길어서 검색 정확도가 떨어짐
    - 작은 단위로 나누면 관련 부분만 정확히 찾을 수 있음

    overlap: 청크 간 겹치는 부분
    - 문맥이 끊기지 않도록 조금씩 겹치게 자름
    """
    chunks = []
    chunk_id = 0

    for page_data in pages_data:
        text = page_data['text']
        page_num = page_data['page_num']

        # 청킹 수행
        start = 0
        while start < len(text):
            end = start + chunk_size
            chunk_text = text[start:end]

            if chunk_text.strip():  # 빈 청크 제외
                chunks.append({
                    "id": f"law_chunk_{chunk_id}",
                    "text": chunk_text,
                    "page": page_num,
                    "start_pos": start
                })
                chunk_id += 1

            start = end - overlap

    return chunks

# 청킹 실행
print(f"\n[진행중] chunk_size=500, overlap=50으로 청킹 중...")
law_chunks = chunk_text(fire_law_pages, chunk_size=500, overlap=50)

print(f"[완료] 총 {len(law_chunks)}개 청크 생성")
print(f"\n[샘플] 첫 번째 청크:")
print(f"ID: {law_chunks[0]['id']}")
print(f"페이지: {law_chunks[0]['page']}")
print(f"길이: {len(law_chunks[0]['text'])} 문자")
print(f"내용: {law_chunks[0]['text'][:200]}...")




[실습 2] 텍스트 청킹 - 작은 단위로 분할

[진행중] chunk_size=500, overlap=50으로 청킹 중...
[완료] 총 45개 청크 생성

[샘플] 첫 번째 청크:
ID: law_chunk_0
페이지: 1
길이: 67 문자
내용: 발 간 등 록 번 호
소방기본법
2025. 6. 21
법제처 국가법령정보센터
Korea Law Service Center...


In [5]:

# ============================================================
# 실습 3: 임베딩 생성
# ============================================================

print("\n\n" + "="*60)
print("[실습 3] 임베딩 - 텍스트를 숫자 벡터로 변환")
print("="*60)

def get_embedding(text: str) -> List[float]:
    """
    텍스트를 벡터로 변환

    임베딩이란?
    - 텍스트를 의미를 담은 숫자 배열로 변환
    - 의미가 비슷한 텍스트는 비슷한 숫자 배열을 가짐
    - 예: "소화기"와 "화재진압장비"는 벡터상 가까이 위치
    """
    response = client.embeddings.create(
        model="text-embedding-3-small",
        input=text
    )
    return response.data[0].embedding

# 임베딩 테스트
print("\n[테스트 1] 단어 임베딩")
sample_text = "소방시설 설치 기준"
sample_embedding = get_embedding(sample_text)
print(f"텍스트: {sample_text}")
print(f"임베딩 차원: {len(sample_embedding)}")
print(f"벡터 샘플 (처음 5개): {sample_embedding[:5]}")

print("\n[테스트 2] 의미 유사도 확인")
text1 = "화재 발생"
text2 = "불이 났어요"
text3 = "소방차"

emb1 = get_embedding(text1)
emb2 = get_embedding(text2)
emb3 = get_embedding(text3)

# 코사인 유사도 계산
import numpy as np

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

print(f"\n'{text1}' vs '{text2}' 유사도: {cosine_similarity(emb1, emb2):.4f} (높음 - 비슷한 의미)")
print(f"'{text1}' vs '{text3}' 유사도: {cosine_similarity(emb1, emb3):.4f} (낮음 - 다른 의미)")





[실습 3] 임베딩 - 텍스트를 숫자 벡터로 변환

[테스트 1] 단어 임베딩
텍스트: 소방시설 설치 기준
임베딩 차원: 1536
벡터 샘플 (처음 5개): [-0.016582194715738297, 0.024550260975956917, 0.0479041151702404, -0.039601054042577744, -0.0019770616199821234]

[테스트 2] 의미 유사도 확인

'화재 발생' vs '불이 났어요' 유사도: 0.2933 (높음 - 비슷한 의미)
'화재 발생' vs '소방차' 유사도: 0.3014 (낮음 - 다른 의미)


In [6]:
# ============================================================
# 실습 4: ChromaDB에 저장
# ============================================================

print("\n\n" + "="*60)
print("[실습 4] Vector Database 구축 - ChromaDB")
print("="*60)

# ChromaDB 클라이언트 생성
print("\n[진행중] ChromaDB 초기화...")
chroma_client = chromadb.Client(Settings(
    persist_directory="./chroma_db",
    anonymized_telemetry=False
))

# 기존 컬렉션 삭제
try:
    chroma_client.delete_collection("fire_law")
except:
    pass

# 새 컬렉션 생성
fire_law_collection = chroma_client.create_collection(
    name="fire_law",
    metadata={"description": "소방기본법 일부"}
)

print("컬렉션 생성 완료")

# 청크 임베딩 및 저장
print("\n[진행중] 청크 임베딩 및 저장 중...")
print("(시간이 좀 걸립니다 - 약 1-2분)")

for i, chunk in enumerate(law_chunks):
    if i % 10 == 0 and i > 0:
        print(f"  진행: {i}/{len(law_chunks)}")

    # 임베딩 생성
    embedding = get_embedding(chunk['text'])

    # ChromaDB에 저장
    fire_law_collection.add(
        ids=[chunk['id']],
        embeddings=[embedding],
        documents=[chunk['text']],
        metadatas=[{
            "page": chunk['page'],
            "source": "소방기본법"
        }]
    )

print(f"[완료] {len(law_chunks)}개 청크 저장 완료!")





[실습 4] Vector Database 구축 - ChromaDB

[진행중] ChromaDB 초기화...
컬렉션 생성 완료

[진행중] 청크 임베딩 및 저장 중...
(시간이 좀 걸립니다 - 약 1-2분)
  진행: 10/45
  진행: 20/45
  진행: 30/45
  진행: 40/45
[완료] 45개 청크 저장 완료!


In [7]:
# ============================================================
# 실습 5: RAG 검색 및 답변 생성
# ============================================================

print("\n\n" + "="*60)
print("[실습 5] RAG 시스템 - 검색 후 답변 생성")
print("="*60)

def search_fire_law(query: str, n_results: int = 3) -> List[Dict]:
    """
    소방기본법 검색 함수

    1. 질문을 임베딩으로 변환
    2. Vector DB에서 유사한 청크 검색
    3. 가장 관련 있는 상위 n개 반환
    """
    query_embedding = get_embedding(query)

    results = fire_law_collection.query(
        query_embeddings=[query_embedding],
        n_results=n_results
    )

    search_results = []
    for i in range(len(results['documents'][0])):
        search_results.append({
            "text": results['documents'][0][i],
            "page": results['metadatas'][0][i]['page'],
            "source": results['metadatas'][0][i]['source']
        })

    return search_results

def answer_with_rag(question: str) -> str:
    """
    RAG 기반 답변 생성

    프로세스:
    1. 질문 분석
    2. 관련 문서 검색 (Vector DB)
    3. 검색된 문서를 컨텍스트로 LLM에 전달
    4. LLM이 문서 기반 답변 생성
    """
    # 1단계: 관련 문서 검색
    search_results = search_fire_law(question, n_results=3)

    # 2단계: 컨텍스트 구성
    context = "\n\n".join([
        f"[출처: {r['source']}, {r['page']}페이지]\n{r['text']}"
        for r in search_results
    ])

    # 3단계: LLM에게 답변 요청
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": """당신은 소방법령 전문가입니다.
제공된 문서를 기반으로 정확하게 답변하세요.
문서에 없는 내용은 "제공된 자료에서 찾을 수 없습니다"라고 하세요."""
            },
            {
                "role": "user",
                "content": f"""다음 문서를 참고하여 질문에 답하세요.

[참고 문서]
{context}

[질문]
{question}"""
            }
        ],
        temperature=0.3,
        max_tokens=500
    )

    return response.choices[0].message.content





[실습 5] RAG 시스템 - 검색 후 답변 생성


In [8]:
# ============================================================
# 최종 테스트
# ============================================================

print("\n[최종 테스트] RAG 시스템 실행")
print("="*60)

test_questions = [
    "소방시설 설치 대상은 어떻게 되나요?",
    "소방시설 점검은 언제 해야 하나요?",
    "위반 시 과태료는 얼마인가요?"
]

for q in test_questions:
    print(f"\n[질문] {q}")
    print("-"*50)

    # 검색 결과 확인
    results = search_fire_law(q, n_results=2)
    print("[검색된 문서]")
    for i, r in enumerate(results, 1):
        print(f"  {i}. {r['source']} {r['page']}페이지")
        print(f"     {r['text'][:150]}...")

    # 답변 생성
    print("\n[답변]")
    answer = answer_with_rag(q)
    print(answer)
    print("="*60)




[최종 테스트] RAG 시스템 실행

[질문] 소방시설 설치 대상은 어떻게 되나요?
--------------------------------------------------
[검색된 문서]
  1. 소방기본법 8페이지
     다. 다만, 「수도법」 제45조에 따라 소화전을 설치하는 일반수도사업자는 관할 소
방서장과 사전협의를 거친 후 소화전을 설치하여야 하며, 설치 사실을 관할 소방서장에게 통
지하고, 그 소화전을 유지ㆍ관리하여야 한다. <개정 2007. 4. 11., 2011. 3. 8....
  2. 소방기본법 13페이지
     1. 시장지역
2. 공장ㆍ창고가 밀집한 지역
3. 목조건물이 밀집한 지역
4. 위험물의 저장 및 처리시설이 밀집한 지역
5. 석유화학제품을 생산하는 공장이 있는 지역
6. 그 밖에 시ㆍ도의 조례로 정하는 지역 또는 장소
[전문개정 2011. 5. 30.]
제20조(관계...

[답변]
제공된 자료에서 소방시설 설치 대상에 대한 구체적인 내용은 명시되어 있지 않습니다. 따라서 "제공된 자료에서 찾을 수 없습니다"라고 말씀드리겠습니다.

[질문] 소방시설 점검은 언제 해야 하나요?
--------------------------------------------------
[검색된 문서]
  1. 소방기본법 4페이지
     「소방기본법」
소방기본법
[시행 2024. 7. 31.] [법률 제20156호, 2024. 1. 30., 일부개정]
소방청 (대응총괄과-총괄,소방활동,소방력동원) 044-205-7562,7572
소방청 (119종합상황실-119종합상황실운영) 044-205-7082
소방...
  2. 소방기본법 7페이지
      제출하여야 하며, 세부계획에 따
른 소방업무를 성실히 수행하여야 한다.<개정 2015. 7. 24., 2017. 7. 26.>
⑤ 소방청장은 소방업무의 체계적 수행을 위하여 필요한 경우 제4항에 따라 시ㆍ도지사가 제
출한 세부계획의 보완 또는 수정을 요청할 수 있다....

[답변]
제공된 자료에서 

In [9]:
# ============================================================
# Lab 2 완료
# ============================================================

print("\n" + "="*60)
print("[완료] Lab 2 완료!")
print("="*60)
print("""
학습 내용:
1. PDF 텍스트 추출 (pdfplumber)
2. 텍스트 청킹 (Chunking)
3. 임베딩 생성 (Embedding)
4. Vector DB 구축 (ChromaDB)
5. RAG 검색 및 답변 생성

핵심 함수:
- search_fire_law(): 소방기본법 검색
- answer_with_rag(): RAG 기반 답변

다음 Lab:
- Lab 3에서는 화재예방 가이드를 추가하여 다중 문서 검색 시스템을 구축합니다.
""")


[완료] Lab 2 완료!

학습 내용:
1. PDF 텍스트 추출 (pdfplumber)
2. 텍스트 청킹 (Chunking)
3. 임베딩 생성 (Embedding)
4. Vector DB 구축 (ChromaDB)
5. RAG 검색 및 답변 생성

핵심 함수:
- search_fire_law(): 소방기본법 검색
- answer_with_rag(): RAG 기반 답변

다음 Lab:
- Lab 3에서는 화재예방 가이드를 추가하여 다중 문서 검색 시스템을 구축합니다.

